# Stanford bunny example

Shows how to use pygeodesic to compute geodesic distances using the Stanford Bunny as an example.

Uses VTK to visualise the mesh, the geodesic distance and the geodesic path.

In [1]:
# Imports
import pygeodesic
import pygeodesic.geodesic as geodesic
import numpy as np
import vtk
from vtk_helpers import *

In [2]:
pygeodesic.__version__

'0.1.6'

## Compute geodesic distance and path between 2 points

In [3]:
# Read the mesh to get the points and faces of the mesh
filename = r'data/stanford_bunny.ply'
reader = vtk.vtkPLYReader()
reader.SetFileName(filename)
reader.Update()
polydata = reader.GetOutput()
points, faces = getPointsAndCellsFromPolydata(polydata)

In [4]:
points, faces

(array([[-0.0378297 ,  0.12794   ,  0.00447467],
        [-0.0447794 ,  0.128887  ,  0.00190497],
        [-0.0680095 ,  0.151244  ,  0.0371953 ],
        ...,
        [-0.0704544 ,  0.150585  , -0.0434585 ],
        [-0.0310262 ,  0.15372799, -0.00354608],
        [-0.0400442 ,  0.15362   , -0.00816685]]),
 array([[21216, 21215, 20399],
        [ 9186,  9280, 14838],
        [16020, 13433,  5187],
        ...,
        [17346, 34909, 17279],
        [34909, 17346, 17277],
        [17277, 17346, 17345]]))

In [5]:
# Initialise the PyGeodesicAlgorithmExact class instance
geoalg = geodesic.PyGeodesicAlgorithmExact(points, faces)

In [6]:
# Get the distance and the path between the source and target points
sourceIndex = 14558
targetIndex = 13039
distance, path = geoalg.geodesicDistance(sourceIndex, targetIndex)

In [7]:
# The geodesic distance between the source and target points
distance

0.22686869313968416

In [8]:
# The geodesic path, represented by a number of 3D points along the path
path

array([[ 0.0602499 ,  0.0678328 ,  0.0161531 ],
       [ 0.05966812,  0.06917454,  0.01614467],
       [ 0.05965691,  0.06919319,  0.01614385],
       ...,
       [-0.01080809,  0.17986084, -0.02632301],
       [-0.01076728,  0.17993033, -0.02646636],
       [-0.0105893 ,  0.180273  , -0.0272114 ]])

## Compute geodesic distance between source point and all other points

In [9]:
source_indices = np.array([sourceIndex])
distances, best_source = geoalg.geodesicDistances(source_indices)

In [10]:
distances, best_source

(array([0.08835966, 0.08197761, 0.09403253, ..., 0.11232286, 0.04478607,
        0.04779301]),
 array([0, 0, 0, ..., 0, 0, 0]))

## Visualise path using VTK

In [11]:
# Create actors
polydata_actor = createPolyDataActor(polydataFromPointsAndCells(points, faces))
path_actor = createPolyLineActor(path, color=(1,1,1))
point_actors = [createSphereActor(points[indx], radius=0.001) for indx in [sourceIndex, targetIndex]]

In [12]:
# Add "distances" to polydata_actor to visualise distance contour from source point
result = polydata_actor.GetMapper().GetInput().GetPointData().SetScalars(nps.numpy_to_vtk(distances))
dmin = distances[np.where(distances != np.inf)].min()
dmax = distances[np.where(distances != np.inf)].max()
polydata_actor.GetMapper().SetScalarRange([dmin, dmax])

In [13]:
# Show VTK render window
v = Viewer()
v.addActors([polydata_actor, path_actor, *point_actors])
v.show()